# Исследовательский проект по НИС «Анализ данных в Python»

## 2 часть: сбор данных о топ 100 фильмах за 2021 год с сайта kinopoisk.ru, отсортированных по кол-ву оценок

#### Ссылка на рейтинг фильмов: https://www.kinopoisk.ru/lists/navigator/2021/?quick_filters=films&tab=all

#### 1. Импорт необходимых библиотек

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import numpy as np 
import pandas as pd
import time

#### 2. Разработка методов для функциональности кода

2.1. Получение страницы в формате BeautifulSoup по её ссылке

In [2]:
def get_beautiful_soup_of_page(link_page):
    page = requests.get(link_page)
    if page.ok:
        page.ecoding="utf-8"
        return bs(page.text, 'html.parser')

2.2. Получение всех коротких ссылок на фильмы со страницы kinopoisk (на одной странице содержатся 50-53 ссылки на фильмы)

In [3]:
def get_all_links_to_films_on_page(page_bs):
    part = page_bs.html.body.find_all("a")
    links = []
    for el in part:
        # возможно исключение при извлечении "href" у атрибута "a"
        try:
            if el["href"].split("/")[1] == "film" and len(el["href"].split("/")) == 4:
                links.append(el["href"])
        except: KeyError
    return links

#### 3. Загрузка страниц и их представление в формате BeautifulSoup

In [16]:
page1 = get_beautiful_soup_of_page("https://www.kinopoisk.ru/lists/navigator/2021/?quick_filters=films&tab=all")
page2 = get_beautiful_soup_of_page("https://www.kinopoisk.ru/lists/navigator/2021/?page=2&quick_filters=films&tab=all")

bs4.BeautifulSoup

#### 4. Получение всех ссылок фильмов из заданной страницы

In [5]:
links = []
links.append(get_all_links_to_films_on_page(page1))
links.append(get_all_links_to_films_on_page(page2))

[['/film/1318972/',
  '/film/1318972/',
  '/film/409424/',
  '/film/409424/',
  '/film/1387021/',
  '/film/1387021/',
  '/film/1199100/',
  '/film/1199100/',
  '/film/804662/',
  '/film/804662/',
  '/film/1109271/',
  '/film/1109271/',
  '/film/1219149/',
  '/film/1219149/',
  '/film/1227967/',
  '/film/1227967/',
  '/film/1355139/',
  '/film/1355139/',
  '/film/1015471/',
  '/film/1015471/',
  '/film/1309596/',
  '/film/1309596/',
  '/film/1445143/',
  '/film/1445143/',
  '/film/1008477/',
  '/film/1008477/',
  '/film/1309570/',
  '/film/1309570/',
  '/film/4419497/',
  '/film/4419497/',
  '/film/823956/',
  '/film/823956/',
  '/film/1045583/',
  '/film/1045583/',
  '/film/1322324/',
  '/film/1322324/',
  '/film/462059/',
  '/film/462059/',
  '/film/1379090/',
  '/film/1379090/',
  '/film/4317395/',
  '/film/4317395/',
  '/film/1009142/',
  '/film/1009142/',
  '/film/1338480/',
  '/film/1338480/',
  '/film/964318/',
  '/film/964318/',
  '/film/1272246/',
  '/film/1272246/',
  '/film/4

#### 5. Подсчёт кол-ва извлечённых ссылок на каждой странице

In [6]:
print(len(links[0]))
len(links[1])

100


100

Если на второй странице будет 52 ссылки на фильмы, то нужно удалить 2 последние

In [7]:
if (len(links[1]) > 50):
    del links[1][-2:]
len(links[1])

98

#### 6. Дополнение до корректных ссылок на фильмы

In [8]:
for i in range(len(links)):
    for j in range(len(links[i])):
        links[i][j] = "http://www.kinopoisk.ru" + links[i][j]
        
links

[['http://www.kinopoisk.ru/film/1318972/',
  'http://www.kinopoisk.ru/film/1318972/',
  'http://www.kinopoisk.ru/film/409424/',
  'http://www.kinopoisk.ru/film/409424/',
  'http://www.kinopoisk.ru/film/1387021/',
  'http://www.kinopoisk.ru/film/1387021/',
  'http://www.kinopoisk.ru/film/1199100/',
  'http://www.kinopoisk.ru/film/1199100/',
  'http://www.kinopoisk.ru/film/804662/',
  'http://www.kinopoisk.ru/film/804662/',
  'http://www.kinopoisk.ru/film/1109271/',
  'http://www.kinopoisk.ru/film/1109271/',
  'http://www.kinopoisk.ru/film/1219149/',
  'http://www.kinopoisk.ru/film/1219149/',
  'http://www.kinopoisk.ru/film/1227967/',
  'http://www.kinopoisk.ru/film/1227967/',
  'http://www.kinopoisk.ru/film/1355139/',
  'http://www.kinopoisk.ru/film/1355139/',
  'http://www.kinopoisk.ru/film/1015471/',
  'http://www.kinopoisk.ru/film/1015471/',
  'http://www.kinopoisk.ru/film/1309596/',
  'http://www.kinopoisk.ru/film/1309596/',
  'http://www.kinopoisk.ru/film/1445143/',
  'http://www.k

#### 7. Парсинг данных

7.1. Функция для упрощения читабельности и уменьшения объёма кода

In [9]:
def repeat_code(column, el):
    # Добавление нового ключа в словрь
    if not(column in data):
        data[column] = list()
        
    # Извлечение из колонок "Бюджет", "Сборы в США", "Сборы в России" денег в целочисленном формате
    if (column in ["Бюджет", "Сборы в США", "Сборы в России"]):
        data[column].append(int(''.join(get_a_text(el)[1:].split())))
        
    # Извлечение возрастного ограничения по MPAA
    elif column == "Рейтинг MPAA":
        data[column].append(get_a_text(el)[0])
        
    # Извлечение сборов в мире
    elif column == "Сборы в мире":
        
        # Обработка ситуации, когда попадается некорректная дата (помимо числа, разделённого пробелами, со знаком $ 
        # присутствуют другие знаки)
        if get_a_text(el)[0] == "+":
            data[column].append(int(''.join(get_a_text(el).split("= $")[1].split())))
            
        # Обработка корректной даты
        else:
            data[column].append(int(''.join(get_a_text(el)[1:].split())))
            
    # Дата из колонок "Время", "Жанр", "Слоган" извлекается оттуда же, откуда мы получаем информацию о колонке (из div)
    elif column != "Время" and column != "Жанр" and column != "Слоган":
        data[column].append(get_a_text(el))

7.2. Получение информации о каком-либо параметре фильма из атрибута "a"

In [10]:
def get_a_text(el):
    try: 
        return el.a.text
    except:
        return ""

7.3. Получение параметра фильма и некоторой информации для определённых параметров фильмов ("Слоган", "Время", "Жанр")

In [11]:
def get_column(el):
    try: 
        return el.div.text
    except:
        return ""

7.4. Извлечём из каждого фильма название, год производства, страну, жанры, режиссёра, продюссеров, операторов, композиторов, художников, монтажеров, бюджет, сборы в США, сборы в мире, сборы в России, премьеру в мире, возрастные ограничения, рейтинг по возрасту согласно МРАА, время в минутах

In [18]:
data = {}

quantity_of_rows = 0

for links_of_page in links:
    for link in links_of_page:
        page3 = get_beautiful_soup_of_page(link)
        print(page3)
        
        # Получение всех div для дальнейшего поиска нужного div
        divs = page3.html.body.find_all("div")
        
        # Поиск нужного div
        for el in divs:
            # Возможно исключение так, как не у всех div есть data-test-id
            try:
                if el["data-test-id"] == "encyclopedic-table":
                    part5 = el
            except: KeyError
         
        # Извлечение названия фильма 
        for el in page3.html.body.find_all("span"):
            try:
                if el["data-tid"] == "75209b22":
                    if not("Название фильма") in data:
                        data["Название фильма"] = list()
                    data["Название фильма"].append(el.text.split(" (")[0])
            except:
                continue
        
        # Получение потенциальных параметров фильма и информации о времени, слогане, жанрах фильма
        info = part5.find_all("div")
        
        # Информация о последней колонки нужна для того, чтобы задавать верные значения для параметров фильма
        last_column = ""
        
        # Получение информации и фильме
        for el in info:
            column = get_column(el)
            # отбор корректных параметров фильма
            if column != "" and column != "—" and column != "Зрители":
                if not(last_column in ["Время", "Жанр", "Слоган"]):
                    repeat_code(column, el)
                
                # Извлечение информации об оставшихся параметров фильма
                elif last_column == "Время":
                    data[last_column].append(int(get_column(el).split()[0]))
                elif last_column == "Жанр":
                    data[last_column].append(get_column(el).split(", "))
                elif last_column == "Слоган" and get_column(el) != "Режиссер":
                    data[last_column].append(get_column(el))
                else:
                    repeat_code(column, el)

                last_column = column
        
        # Подсчёт извлечённых фильмов. Если в одной из колонок окажется меньше информации, чем текущее кол-во строк, то словарь
        # не преобразуется в DataFrame => не преобразуется в .xlsx файл  
        quantity_of_rows += 1
        
        # Проверка корректности кол-ва строк в каждом столбце
        for key in data.keys():
            if len(data[key]) < quantity_of_rows:
                data[key].append(np.NaN)
        
data

<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#"><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><title data-react-helmet="true">ÐÐ¹!</title><meta content="Ð¯Ð½Ð´ÐµÐºÑ" data-react-helmet="true" property="og:title"/><meta content="ÐÐ°Ð¹Ð´ÑÑÑÑ Ð²ÑÑ" data-react-helmet="true" property="og:description"/><meta content="https://yastatic.net/s3/home-static/_/37/37a02b5dc7a51abac55d8a5b6c865f0e.png" data-react-helmet="true" property="og:image"/><link href="/captcha_smart.0bfc4d9561e7e6e94795.min.css?k=1666276736660" rel="stylesheet"/><style>@media only screen and (min-width:651px){body{background-image:url('https://captcha-backgrounds.s3.yandex.net/static/kinopoisk-background.jpg')}}.LogoLink{background-image:url('')}</style><div id="root"><div class="Theme Theme_color_yandex-default Theme_root_default"><div class="Container"><div class="Spacer" style="padding-bottom:40px"><a class="Lin

AttributeError: 'NoneType' object has no attribute 'find_all'

#### 8. Удаление ненужных столбцов

In [ ]:
del data["Маркетинг"]
data

#### 9. Проверка корректности кол-ва элементов в каждом столбце (должно быть 100)

In [ ]:
for key in data.keys():
    if len(data[key]) == 100:
        print("yes")
    else:
        print("no")

#### 10. Преобразование получившегося словаря с данными в DataFrame

In [ ]:
date = pd.DataFrame.from_dict(data)
date

#### 11. Изменение имён некоторых целочисленных столбцов для понимания, о какой СИ идёт речь 

In [ ]:
date = date.rename(columns={"Бюджет" : "Бюджет в $", "Сборы в США" : "Сборы в США в $", "Сборы в мире" : "Сборы в мире в $", 
                            "Сборы в России" : "Сборы в России в $", "Время" : "Время в мин"})
date

#### 11. Конвертирование DataFrame в excel таблицу формата .xlsx

In [ ]:
writer = pd.ExcelWriter('Top100FilmsFromKinopoiskBy2021Year.xlsx')
date.to_excel(writer,'Sheet5')
writer.save()

#### 12. Проверка корректности получившегося датасета 

In [ ]:
data = pd.read_excel("Top100FilmsFromKinopoiskBy2021Year.xlsx")
data